<a href="https://colab.research.google.com/github/lisethrubio/Project4-Machine_Learning_Model/blob/Steven_branch2/Health_Prediction_Model_Class_x_12dim_inc_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

In [2]:
# Import our input dataset
body_df = pd.read_csv('/bodyPerformance.csv')
body_df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [3]:
# Encode (convert to dummy variables) the "gender" column
gender_encoded = pd.get_dummies(body_df['gender'], prefix="gender", dtype=int)

In [4]:
gender_encoded.head()

,gender_F,gender_M
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [5]:
# Concatenate the scaled data DataFrame with the "IMF Country Code" encoded dummies
new_body_df = pd.concat([body_df, gender_encoded], axis=1)


In [6]:
# Encode the target variable 'class'
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(new_body_df["class"].values)

# Remove 'class' and 'gender' columns from the features data
X = new_body_df.drop(columns=["class", "gender"]).values

In [7]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [8]:
# Preprocess numerical data for the neural network
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=4)

In [10]:
# Define the deep learning model
nn_model = tf.keras.models.Sequential()

# Update input_dim to match the number of input features (10)
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=12))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer for 4 classes with softmax activation
nn_model.add(tf.keras.layers.Dense(units=4, activation="softmax"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=155)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3621 - loss: 1.2823
Epoch 2/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5836 - loss: 0.9367
Epoch 3/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6147 - loss: 0.8600
Epoch 4/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6249 - loss: 0.8315
Epoch 5/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6418 - loss: 0.8133
Epoch 6/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6487 - loss: 0.7959
Epoch 7/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6590 - loss: 0.7765
Epoch 8/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6753 - loss: 0.7571
Epoch 9/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6795 - loss: 0.7517
Epoch 10/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6950 - loss: 0.7268
Epoch 11/155
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7099 - loss: 0.6947
Epoch 12/155
314/314 ━━━━━━━━━━━━━━━━━━━━

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - 3ms/step - accuracy: 0.7537 - loss: 0.6219
Loss: 0.6218504309654236, Accuracy: 0.7536578178405762


In [12]:
# Export our model to HDF5 file
from google.colab import files
nn_model.save('Health_Prediction_Model_Class_x_12dim_inc_gender.h5')
files.download('Health_Prediction_Model_Class_x_12dim_inc_gender.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>